#NOTA:
**Este notebook sirve para llevar a la estructura requerida para entrenar yolov6, tanto de los set de trainig, como de los set de testing, como de los set de validation. Para hacerlo para cada uno de estos sets, basta con cambiar algunos paths.**

In [1]:
#Enlazamos con drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Librerias útiles:
import cv2
import numpy as np
import shutil #Libreria parea manipularv archivos, carpetas, directorios, etc...
import os  #Libreria parea manipularv archivos, carpetas, directorios, etc...
import pandas as pd

In [3]:
os.mkdir("trainin") #Creamos la carpeta trainin
os.mkdir("testin") #Creamos la carpeta testin
os.mkdir("val") #Creamos la carpeta de val

In [4]:
!unzip "/content/drive/MyDrive/Yolo_v6/datasets_usados_para_entrenar/VisDrone2019-VID-test-dev.zip" -d "/content/testin" #Descomprimimos el dataset de testing en la carpeta testin creada

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000009.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000010.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000011.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000012.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000013.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000014.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000015.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000016.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/sequences/uav0000088_00290_v/0000017.jpg  
  inflating: /content/testin/VisDrone2019-VID-test-dev/se

In [ ]:
!unzip "/content/drive/MyDrive/Yolo_v6/datasets_usados_para_entrenar/VisDrone2019-VID-train.zip" -d  "/content/trainin" #Descomprimimos el dataset de training en la carpeta trainin creada

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000145.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000146.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000147.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000148.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000149.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000150.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000151.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000152.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_00000_v/0000153.jpg  
  inflating: /content/trainin/VisDrone2019-VID-train/sequences/uav0000308_0

In [ ]:
!unzip "/content/drive/MyDrive/Yolo_v6/datasets_usados_para_entrenar/VisDrone2019-VID-val.zip" -d "/content/val" #Descomprimimos el dataset de val en la carpeta val creada

Archive:  /content/drive/MyDrive/Yolo_v6/datasets_usados_para_entrenar/VisDrone2019-VID-val.zip
   creating: /content/val/VisDrone2019-VID-val/
   creating: /content/val/VisDrone2019-VID-val/annotations/
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000086_00000_v.txt  
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000117_02622_v.txt  
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000137_00458_v.txt  
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000182_00000_v.txt  
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000268_05773_v.txt  
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000305_00000_v.txt  
  inflating: /content/val/VisDrone2019-VID-val/annotations/uav0000339_00001_v.txt  
   creating: /content/val/VisDrone2019-VID-val/sequences/
   creating: /content/val/VisDrone2019-VID-val/sequences/uav0000086_00000_v/
  inflating: /content/val/VisDrone2019-VID-val/sequences/uav0000086_00000

In [5]:
os.mkdir("custom_dataset") #Se crea carpeta custom_dataset
os.makedirs(os.path.join("custom_dataset", "images")) #Se crea carpeta images en custom_dataset
os.makedirs(os.path.join("custom_dataset", "labels")) #Se crea carpeta labels en custom_dataset

In [6]:
os.makedirs(os.path.join("custom_dataset/labels", "test")) #Se crea carpeta train en custom_dataset/labels
os.makedirs(os.path.join("custom_dataset/images", "test")) #Se crea carpeta train en custom_dataset/images


#PROCESO PARA LLEVAR LAS IMAGENES A SU RESPECTIVA CARPETA:

In [7]:
sub_images=os.listdir('/content/testin/VisDrone2019-VID-test-dev/sequences') #Lista con los nombres de las carpetas que contienen las imagenes de entrenamiento

In [8]:
num_sub_images=[] #Lista con el número de imagenes que tiene cada carpeta
name_sub_images=[] #Lista que contiene las listas con los nombres de las imagenes de cada carpeta
for i in sub_images:
  num_sub_images.append(len(os.listdir('/content/testin/VisDrone2019-VID-test-dev/sequences'+"/"+i)))
  name_sub_images.append(os.listdir('/content/testin/VisDrone2019-VID-test-dev/sequences'+"/"+i))

In [9]:
init_path="/content/testin/VisDrone2019-VID-test-dev/sequences" #Path inicial
n_jpg=[]
#Se cambia el nombre a las imagenes(concatenandoles el nombre de su respectiva carpeta) y se copian TODAS las imagenes en la carpeta custom_dataset/images/train
for i in range(len(sub_images)):
  init_pathh=init_path+"/"+sub_images[i]
  for j in range(len(name_sub_images[i])):
    init_pathhh=init_pathh+"/"+name_sub_images[i][j]
    os.rename(init_pathhh,init_pathhh.replace(".jpg",sub_images[i]+".jpg"))
    shutil.copy(init_pathhh.replace(".jpg",sub_images[i]+".jpg"),"/content/custom_dataset/images/test")
    n_jpg.append(name_sub_images[i][j].replace(".jpg",sub_images[i]+".jpg"))
    

#PROCESO PARA LLEVAR LAS ANOTACIONES A SU RESPECTIVA CARPETA:

In [10]:
names_videos=os.listdir('/content/testin/VisDrone2019-VID-test-dev/annotations') #Nombres de los archivos que contienen las anotaciones de las imagenes de cada video

In [11]:
os.mkdir("anotaciones") #Se crea carpeta para guardar los archivos txt una vez se reemplazaron las comas por espacios

#Cambiando las comas por espacios en los archivos txt:
for i in range(len(names_videos)):
  fin = open("/content/testin/VisDrone2019-VID-test-dev/annotations/"+names_videos[i], "rt") #Path donde se encuentra el archivo txt
  fout = open("/content/anotaciones/"+names_videos[i], "wt")  #Path donde quiero que guarde el archivo txt modificado

#for each line in the input file
  for line in fin:
    #read replace the string and write to output file
    fout.write(line.replace(',', ' ')) #Reemplazando comas por espacios
  #close input and output files
  fin.close()
  fout.close()

In [12]:
videos=[] #Lista donde estarán los archivos txt que contienen las anotaciones de las imagenes de los videos
for i in range(len(names_videos)):
  videos.append(pd.read_csv('/content/anotaciones'+"/"+names_videos[i], sep=" ",header=None)) #Voy agragando los archivos txt a la lista videos
  videos[i]=np.asfarray(videos[i].to_numpy())  #Paso los dataframe de la lista videos a ndarray y que los elementos queden en formato flotante

#Formato de las Anotaciones:
En los dataset trabajados las anotaciones de las imagenes estaban en 10 columnas, las cuales se pueden ver en:
https://github.com/VisDrone/VisDrone2018-VID-toolkit

Pero se necesita que el formato de las anotaciones esté en 5 columnas; por tanto se procede a hacer esto. El formato de las anotaciones requeridas se puede ver en:
https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/

In [13]:
a=pd.read_csv('/content/anotaciones'+"/"+names_videos[6], sep=" ",header=None)
a

,0,1,2,3,4,5,6,7,8,9
0,1,0,951,590,22,42,1,1,0,0
1,2,0,952,590,22,42,1,1,0,0
2,3,0,954,590,22,43,1,1,0,0
3,4,0,956,591,22,42,1,1,0,0
4,5,0,958,591,22,43,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
34663,308,162,677,618,18,65,1,2,0,0
34664,309,162,676,618,19,66,1,2,0,0
34665,310,162,675,618,19,67,1,2,0,0
34666,311,162,674,618,20,68,1,2,0,0


In [14]:
len(videos[6])

34668

In [15]:
#Cambiando el formato a las anotaciones y que queden en formato flotante:
for i in range(len(videos)):
  #Resolución de las imagenes de cada video(ya que las imagenes entre los videos no tienen la misma resolución):
  path_images="/content/testin/VisDrone2019-VID-test-dev/sequences/"+names_videos[i].replace(".txt","")
  images_names=os.listdir(path_images)
  high=cv2.imread(path_images+"/"+images_names[0]).shape[0] #High
  width=cv2.imread(path_images+"/"+images_names[0]).shape[1] #width

  videos[i][...,1]=videos[i][...,7] #class_id
  videos[i][...,2]=(videos[i][...,2]+videos[i][...,4]/2)/width #center x normalizado
  videos[i][...,3]=(videos[i][...,3]+videos[i][...,5]/2)/high#center y normalizado
  videos[i][...,4]=(videos[i][...,4])/width#box_width normalizado
  videos[i][...,5]=(videos[i][...,5])/high #box_height normalizado

In [16]:
videos[0][...,4]

array([0.07195378, 0.0677521 , 0.0625    , ..., 0.0157563 , 0.01523109,
       0.01523109])

In [17]:
for i in range(len(videos)):
  videos[i]=np.delete(videos[i],(6,7,8,9),axis=1) #Se eliminan las últimas 4 columnas de las anotaciones (No se requieren)

In [18]:
num_images=[]
for i in range(len(videos)):
  num_images.append(np.unique(videos[i][...,0])) #Anotaciones de imagenes que hay en cada archivo txt

In [19]:
txt_separados_ext={} #En cada llave del diccionario hay un numpy correspondiente a un archivo txt
for i in range(len(videos)):
  txt_separados_ext[i]={} #Dentro de cada llave del diccionario hay otro diccionario, en los cuales cada llave corresponde a las anotaciones de cada una de las respectivas imagenes ya de forma separada
  for j in num_images[i]:
    txt_separados_ext[i][j]=[]

In [20]:
#Ya se arma como tal la estructura que contiene la anotación de cada una de las imagenes de forma separada:
for i in range(len(videos)):
  for j in range(len(videos[i])):
    for k in num_images[i]:
      if videos[i][j][0]==k:
        txt_separados_ext[i][k].append(videos[i][j]) 

In [21]:
for i in range(len(videos)):
  for key in txt_separados_ext[i]:
    txt_separados_ext[i][key]=np.delete(txt_separados_ext[i][key],0,axis=1) #Se elimina la primera columna de cada archivo de anotación; pues ya no se necesita

In [22]:
names_videos[6]

'uav0000073_04464_v.txt'

In [23]:
len(txt_separados_ext[6])

312

In [24]:
txt_separados_ext[6].keys()

dict_keys([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0

In [25]:
#Se llevan nuevamente los numpy a los archivos txt y con su respectivo nombre(Esto se guarda en la carpeta custom_dataset/labels/train)
n_txt=[]
path="/content/custom_dataset/labels/test/"
for j in range(len(txt_separados_ext)):
  name_p=names_videos[j]
  for i in range(1,len(txt_separados_ext[j])+1):
    if i<10:
      name="000000"+str(i)+name_p
    elif (i>=10 and i<100):
      name="00000"+str(i)+name_p
    elif (i>=100 and i<1000):
      name="0000"+str(i)+name_p
    else:
      name="000"+str(i)+name_p
    n_txt.append(name)
    
    #Para que se asegure que la llave está en el diccionario; ya que hay algunas imagenes que no tienen anotación; ya que no detectaron ningún objeto:
    if i in txt_separados_ext[j].keys():
      panda=pd.DataFrame(txt_separados_ext[j][i]) #Se lleva el arreglo a un numpy para tratar con más comodidad
      panda[0]=panda[0].astype(int) #Para que la clase me quede como entero
      for k in range(5):
        panda[k]=np.round(panda[k],6) #Para que redondee todos los valores con seis cifras significativas
      
      #Escribiendo la información en el archivo txt:
      file = open(path+name, "w") #Donde guardaré el archvio y que sea para escritura
      index=panda.columns.values #Nombres de las columnas del dataframe
      for l in range(len(panda.index.values)):
        for m in index:
          if m!=index[len(index)-1]:
            file.write(str(panda[m][l])) #Escribiendo columna por columna en el txt
            file.write(" ") #Entre cada columna hay un espacio
          else:
            file.write(str(panda[m][l])+os.linesep) #Para el salto de linea
      file.close() #Cierro el file txt ya terminado

In [26]:
a=len(os.listdir("/content/custom_dataset/labels/test"))
b=len(os.listdir("/content/custom_dataset/images/test"))
print(a,b) #Veamos que algunas imagenes no tienen su respectiva anotación (Hay algunas anotaciones incompletas)

6635 6635


In [27]:
os.listdir("/content/testin")[0]

'VisDrone2019-VID-test-dev'

In [28]:
%ls /content/drive/MyDrive/Yolo_v6

datasets_usados_estructurados/  exp10/  exp14/  exp5/  exp9/
datasets_usados_para_entrenar/  exp11/  exp2/   exp6/  train/
exp/                            exp12/  exp3/   exp7/  videos_imagenes/
exp1/                           exp13/  exp4/   exp8/


In [29]:
shutil.copytree("/content/custom_dataset","/content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/"+os.listdir("/content/testin")[0]) #LLevamos el dataset estructurado para entrenar al drive

'/content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/VisDrone2019-VID-test-dev'